In [7]:
import numpy as np
import os
import multiprocessing
import time
from functools import partial
from tqdm import notebook

In [8]:
def heavy(size, id, return_dict, arr=None):
    a = []
    for i in range(size):
        a.append(i)
    return_dict[id] = a
    print('Done')

# doesn't work in notebooks
def heavy_queue(size, id, queue):
    a = []
    for i in range(size):
        a.append(i)
    dct = queue.get()
    dct[i] = a
    queue.put(dct)
    print('Done')

# doesn't work
heavy_dict = {}
def heavy_global(size, id):
    a = []
    for i in range(size):
        a.append(i)
    global heavy_dict
    heavy_dict[id] = a
z="""    
def heavy_write(size, id):
    a = []
    for i in range(size):
        a.append(i)
    
    fname = 'dummy' + str(id) + '.txt'
    if fname in os.listdir():
        os.system('rm ' + fname)
    with open('dummy' + str(id) + '.txt', 'w') as f:
        for aval in a:
            f.write(str(aval) + '\n')"""

In [4]:
%%time
size = int(5e7)
return_dict = {}
for i in range(5):
    return_dict[i] = heavy(size, i, {})

Done
Done
Done
Done
Done
CPU times: user 16.9 s, sys: 1.36 s, total: 18.3 s
Wall time: 18.3 s


In [5]:
%%time
for i in range(5):
    heavy_write(size, i)

CPU times: user 9.87 s, sys: 401 ms, total: 10.3 s
Wall time: 10.4 s


In [5]:
%%time
def heavy_simple(id, size=size):  heavy(size, id, {})
p = multiprocessing.Pool(processes=5)
p.map(heavy_simple, range(5))
p.close()
p.join()

Done
Done
DoneDone
Done

CPU times: user 19.4 ms, sys: 38.5 ms, total: 57.8 ms
Wall time: 5.37 s


In [7]:
%%time
# this is the best way, probably
jobs = []
for i in range(5):
    p = multiprocessing.Process(target=heavy, args=(size,i,{}))
    p.start()
    jobs.append(p)

for p in jobs:
    p.join()

Done
Done
Done
Done
Done
CPU times: user 12.4 ms, sys: 24.9 ms, total: 37.3 ms
Wall time: 4.6 s


In [9]:
%%time
# DOESN'T WORK
jobs = []
heavy_dict = {}
for i in range(5):
    p = multiprocessing.Process(target=heavy_global, args=(size,i))
    p.start()
    jobs.append(p)

for p in jobs:
    p.join()
print(len(heavy_dict))

0
CPU times: user 7.26 ms, sys: 19 ms, total: 26.2 ms
Wall time: 467 ms


In [10]:
%%time
# GETS BAD WITH LARGE SIZE -- ALSO MAYBE JUST DOESN'T WORK
with multiprocessing.Manager() as manager:
    return_dict = manager.dict()
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=heavy, args=(size, i, return_dict))
        p.start()
        jobs.append(p)
    for p in jobs:
        p.join()
    print(len(return_dict[0]))

Done
Done
Done
Done
Done
5000000
CPU times: user 578 ms, sys: 684 ms, total: 1.26 s
Wall time: 4.32 s


In [ ]:
%%time
# GETS BAD WITH LARGE SIZE
p = multiprocessing.Pool(processes=5)
return_dict = {}
for i in range(5):
    return_dict[i] = p.apply_async(heavy, [size, i, {}])
p.close()
p.join()
#return_dict = {key: val.get() for key,val in return_dict.items()}
#print(return_dict[0][:10], len(return_dict[4]))

In [161]:
%%time
# DOESN'T WORK IN NOTEBOOKS
queue = multiprocessing.Queue()
queue.put({})
jobs = []
for i in range(5):
    p = multiprocessing.Process(target=heavy_queue, args=(size, i, queue))
    jobs.append(p)
    p.start()

dct = queue.get(timeout=5)
for p in jobs:
    p.join()


Done
Done
Done
Done
Done


Process Process-234:
Traceback (most recent call last):
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/util.py", line 337, in _exit_function
    _run_finalizers()
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/util.py", line 277, in _run_finalizers
    finalizer()
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/util.py", line 201, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 192, in _finalize_join
    thread.join()


KeyboardInterrupt: 

  File "/Users/taylora/opt/anaconda3/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/Users/taylora/opt/anaconda3/lib/python3.7/threading.py", line 1060, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


# Documentation

In [118]:
help(multiprocessing.Value)

Help on method Value in module multiprocessing.context:

Value(typecode_or_type, *args, lock=True) method of multiprocessing.context.DefaultContext instance
    Returns a synchronized shared object



In [103]:
help(multiprocessing.Manager)

Help on method Manager in module multiprocessing.context:

Manager() method of multiprocessing.context.DefaultContext instance
    Returns a manager associated with a running server process
    
    The managers methods such as `Lock()`, `Condition()` and `Queue()`
    can be used to create shared objects.



In [98]:
help(multiprocessing.Process)

Help on class Process in module multiprocessing.context:

class Process(multiprocessing.process.BaseProcess)
 |  Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |  
 |  Process objects represent activity that is run in a separate process
 |  
 |  The class is analogous to `threading.Thread`
 |  
 |  Method resolution order:
 |      Process
 |      multiprocessing.process.BaseProcess
 |      builtins.object
 |  
 |  Methods inherited from multiprocessing.process.BaseProcess:
 |  
 |  __init__(self, group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  close(self)
 |      Close the Process object.
 |      
 |      This method releases resources held by the Process object.  It is
 |      an error to call this method if the child process is still running.
 |  
 |  is_alive(self)
 |      Return whether process i

# Testing tqdm

In [16]:
from tqdm.notebook import tqdm as tqdm
import time

In [48]:
def heavy_tqdm(id, pbar):
    for i in range(10):
        time.sleep(0.1)
        pbar.update()
    print('Done')

def simple_tqdm(id):
    with tqdm(total=10, position=id) as pbar:
        for i in range(10):
        #for i in tqdm(range(10)):
            time.sleep(0.1)
            pbar.update(1)
    print('Done')

In [23]:
#size = int(5e5) # pbar makes it slow
for i in range(5):
    pbar = tqdm(total=10)
    heavy_tqdm(i, pbar)

Done


Done



Done


Done


Done


In [49]:
jobs = []
for i in range(5):
    #pbar = tqdm(total=10)
    p = multiprocessing.Process(target=simple_tqdm, args=(i,))
    p.start()
    jobs.append(p)

for p in jobs:
    p.join()



Done

Done

Done

Done
Done


In [51]:
p = multiprocessing.Pool(5, initargs=(multiprocessing.RLock(),), initializer=tqdm.set_lock)
p.map(simple_tqdm, range(5))
#p.close()
print('\n'*10)


Done

Done


Done
Done

Done













Process ForkPoolWorker-69:
Process ForkPoolWorker-68:
Process ForkPoolWorker-71:
Process ForkPoolWorker-70:
Process ForkPoolWorker-67:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/taylora/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run

In [ ]:
with multiprocessing.Pool(5) as p:
    x = list(tqdm(p.imap(target=heavy_tqdm, range(5)), total=5))

In [24]:
jobs = []
for i in range(5):
    pbar = tqdm(total=10)
    p = multiprocessing.Process(target=heavy_tqdm, args=(i, pbar))
    p.start()
    jobs.append(p)

for p in jobs:
    p.join()

Done
Done
Done
Done
Done


In [70]:
%%time
for i in tasks:
    simple_tqdm(i, None)


CPU times: user 1.64 s, sys: 344 ms, total: 1.98 s
Wall time: 22.1 s


In [73]:
z = tqdm()

In [71]:
%%time
# 4.8s with tqdm
def simple_tqdm(id, lock):
    text = "Job #" + str(id)
    total=100
    
    if lock is None:
        progress = tqdm(total=total, position=id, desc=text)
        for _ in range(0, total, 5):
            progress.update(5)
            time.sleep(0.1)
        progress.close()
    
    else:
        with lock:
            progress = tqdm(total=total, position=id, desc=text)
        for _ in range(0, total, 5):
            with lock:
                progress.update(5)
            time.sleep(0.1)
        with lock:
            progress.close()

#pool = multiprocessing.Pool(5)
pool = ThreadPool(5)
tasks = range(10)
lock = multiprocessing.Lock()
for i in tasks:
    #pool.apply_async(simple_tqdm, args=(i, lock))
    pool.apply_async(simple_tqdm, args=(i, None)) # works the same
pool.close()
pool.join()






CPU times: user 643 ms, sys: 99 ms, total: 742 ms
Wall time: 4.5 s


In [55]:
from multiprocessing.pool import ThreadPool
import time
import threading
#from tqdm import tqdm


def demo(lock, position, total):
    text = "progresser #{}".format(position)
    with lock:
        progress = tqdm(
            total=total,
            position=position,
            desc=text,
        )
    for _ in range(0, total, 5):
        with lock:
            progress.update(5)
        time.sleep(0.1)
    with lock:
        progress.close()


pool = ThreadPool(5)
#pool = multiprocessing.Pool(5)
tasks = range(10)
lock = threading.Lock()
for i, url in enumerate(tasks, 1):
    pool.apply_async(demo, args=(lock, i, 100))
pool.close()
pool.join()

In [ ]:
from multiprocessing import current_process

""" Your code Here
Here, the current_process() is the process object
current_process().name gives the name of the process
current_process()._identity gives a tuple of the number of process
"""

current = current_process()
with tqdm(total=iterator_size) as progress_bar:
    for row in tqdm(batch, desc=str(current.name),
              position=current._identity[0] - 1)):
        process_batch(batch)
        progress_bar.update(1)

In [86]:
from multiprocessing import Process
def process_tqdm(id):
    total = 100
    with tqdm(total=total, position=id, desc=str(id)) as pbar:
        for i in range(0,total,5):
            time.sleep(0.1)
            pbar.update(5)
    print('Done')

jobs = []
tqdm.set_lock(multiprocessing.RLock())
for i in range(5):
    p = Process(target=process_tqdm, args=(i,), initializer=tqdm.set_lock)
    p.start()
    jobs.append(p)

    
for p in jobs:
    p.join()
    
    

TypeError: __init__() got an unexpected keyword argument 'initializer'

# Code blocks for StackOverflow

First question: Why is Pool slower than sequential or Process?

Second question: Why doesn't Process show tqdm?

In [24]:
from multiprocessing import Pool, Process
from multiprocessing.pool import ThreadPool
size = int(5e6)
def foo(id, size=size):
    a = []
    for i in range(size):
        a.append(i)
    print('Done')

In [8]:
%%time
for i in range(5):
    foo(i)

Done
Done
Done
Done
Done
CPU times: user 1.68 s, sys: 118 ms, total: 1.79 s
Wall time: 1.79 s


In [9]:
%%time
p = Pool(5)
p.map_async(foo, range(5))
p.close()
p.join()

Done
DoneDone

Done
Done
CPU times: user 14.1 ms, sys: 23.7 ms, total: 37.8 ms
Wall time: 675 ms


In [10]:
%%time
jobs = []
for i in range(5):
    p = Process(target=foo, args=(i,))
    p.start()
    jobs.append(p)

for p in jobs:
    p.join()

Done
DoneDone

Done
Done
CPU times: user 12.6 ms, sys: 22 ms, total: 34.6 ms
Wall time: 602 ms


In [42]:
from tqdm import notebook
import time
def foo2(id):
    total = 100
    print(' ', end='', flush=True)
    for _ in notebook.tqdm(range(0, total, 5)):
        time.sleep(0.1)
#    with notebook.tqdm(total=total, position=id) as pbar:
#        for _ in range(0, total, 5):
#            pbar.update(5)
#            time.sleep(0.1)

In [43]:
%%time
for i in range(5):
    foo2(i)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

CPU times: user 563 ms, sys: 417 ms, total: 980 ms
Wall time: 10.6 s


In [33]:
import tqdm
tqdm.__version__

'4.57.0'

In [34]:
import multiprocessing

AttributeError: module 'multiprocessing' has no attribute '__version__'

In [38]:
%%time
from multiprocessing.pool import ThreadPool
pool = ThreadPool(5)
pool.map(foo2, range(5))
pool.close()
pool.join()

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 337 ms, sys: 118 ms, total: 455 ms
Wall time: 2.27 s


In [45]:
%%time
pool = Pool(5)
for i in range(5):
    pool.apply_async(foo2, args=(i,))
pool.close()
pool.join()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

CPU times: user 210 ms, sys: 244 ms, total: 453 ms
Wall time: 2.44 s


In [44]:
%%time
from multiprocessing import Pool
pool = Pool(5)
pool.map(foo2, range(5))
pool.close()
pool.join()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

CPU times: user 215 ms, sys: 245 ms, total: 460 ms
Wall time: 2.4 s


In [29]:
%%time
from multiprocessing import Process
jobs = []
for i in range(5):
    p = Process(target=foo2, args=(i,))
    p.start()
    jobs.append(p)

for p in jobs:
    p.join()

CPU times: user 7.08 ms, sys: 16.3 ms, total: 23.4 ms
Wall time: 2.11 s


In [31]:
help(notebook.trange)

Help on function tnrange in module tqdm.notebook:

tnrange(*args, **kwargs)
    A shortcut for `tqdm.notebook.tqdm(xrange(*args), **kwargs)`.
    On Python3+, `range` is used instead of `xrange`.



In [35]:
from time import sleep
from tqdm import trange, auto
from multiprocessing import Pool, RLock, freeze_support

L = list(range(9))

def progresser(n):
    interval = 0.001 / (n + 2)
    total = 5000
    text = "#{}, est. {:<04.2}s".format(n, interval * total)
    for _ in auto.trange(total, desc=text, position=n):
        sleep(interval)

if __name__ == '__main__':
    #freeze_support()  # for Windows support
    auto.tqdm.set_lock(RLock())  # for managing output contention
    p = Pool(initializer=auto.tqdm.set_lock, initargs=(auto.tqdm.get_lock(),))
    p.map(progresser, L)

In [21]:
from time import sleep
import multiprocessing
import tqdm

import logging
from tqdm_multiprocess.logger import setup_logger_tqdm
logger = logging.getLogger(__name__)

from tqdm_multiprocess import TqdmMultiProcessPool

iterations1 = 100
iterations2 = 5
iterations3 = 1
def some_other_function(tqdm_func, global_tqdm):

    total_iterations = iterations1 * iterations2 * iterations3
    with tqdm_func(total=total_iterations, dynamic_ncols=True) as progress3:
        progress3.set_description("outer")
        for i in range(iterations3):
            logger.info("outer")
            total_iterations = iterations1 * iterations2
            with tqdm_func(total=total_iterations, dynamic_ncols=True) as progress2:
                progress2.set_description("middle")
                for j in range(iterations2):
                    logger.info("middle")
                    #for k in tqdm_func(range(iterations1), dynamic_ncols=True, desc="inner"):
                    with tqdm_func(total=iterations1, dynamic_ncols=True) as progress1:
                        for j in range(iterations1):
                            # logger.info("inner") # Spam slows down tqdm too much
                            progress1.set_description("innert")
                            sleep(0.01)
                            progress1.update()
                            progress2.update()
                            progress3.update()
                            global_tqdm.update()

    logger.warning(f"Warning test message. {multiprocessing.current_process().name}")
    logger.error(f"Error test message. {multiprocessing.current_process().name}")


# Multiprocessed
def example_multiprocessing_function(some_input, tqdm_func, global_tqdm):  
    logger.debug(f"Debug test message - I won't show up in console. {multiprocessing.current_process().name}")
    logger.info(f"Info test message. {multiprocessing.current_process().name}")
    some_other_function(tqdm_func, global_tqdm)
    return True

def error_callback(result):
    print("Error!")

def done_callback(result):
    print("Done. Result: ", result)

def example():
    pool = TqdmMultiProcessPool(4)
    process_count = 4
    task_count = 10
    initial_tasks = [(example_multiprocessing_function, (i,)) for i in range(task_count)]    
    total_iterations = iterations1 * iterations2 * iterations3 * task_count
    with notebook.tqdm(total=total_iterations, dynamic_ncols=True) as global_progress:
        global_progress.set_description("global")
        results = pool.map(global_progress, initial_tasks, error_callback, done_callback)
        print(results)

In [23]:
logfile_path = "tqdm_multiprocessing_example.log"
setup_logger_tqdm(logfile_path) # Logger will write messages using tqdm.write
example()

  0%|          | 0/5000 [00:00<?, ?it/s]


























                    


                                                
                                                      

                                                   



                                             




                                          





                                       






                                   







                                 








                              









                          










                       












  0%|          | 0/500 [00:25<?, ?it/s]











innert:  13%|█▎        | 13/100 [01:06<01:19,  1.09it/s]


outer:  11%|█▏        | 113/1000 [01:44<14:01,  1.05it/s]

outer:  11%|█         | 112/1000 [01:44<13:33,  1.09it/s]



middle:  23%|██▎       | 113/500 [01:44<06:08,  1.05it/s]




outer:  11%|█▏        | 113/1000 [01:44<13:34,  1.09it/s]





middle:  22%|██▏       | 112/500 [01:44<05:54,  1.10it/s]






innert:  13%|█▎        | 13/100

INFO - 02/26/21 10:01:53 - 0:04:27 - SIGINT or CTRL-C detected, closing pool. Please wait.
INFO - 02/26/21 10:01:53 - 0:04:27 - Info test message. ForkPoolWorker-39
INFO - 02/26/21 10:01:53 - 0:04:27 - Info test message. ForkPoolWorker-41
INFO - 02/26/21 10:01:53 - 0:04:27 - outer
INFO - 02/26/21 10:01:53 - 0:04:27 - middle


innert:  12%|█▏        | 12/100 [01:06<01:20,  1.10it/s]










innert:  13%|█▎        | 13/100 [01:07<01:21,  1.06it/s]
























                    


                                                
                                                      

                                                   



                                             




                                          





                                       






                                   







                                 








                              









                          










                       












  0%|          | 0/500 [00:25<?, ?it/s]











innert:  13%|█▎        | 13/100 [01:06<01:19,  1.09it/s]


outer:  11%|█▏        | 113/1000 [01:44<14:01,  1.05it/s]

outer:  11%|█         | 112/1000 [01:44<13:33,  1.09it/s]



middle:  23%|██▎       | 113/500 [01:44<06:08,  1.05it/s]




outer:  11%|█▏        | 113/1000 [0

INFO - 02/26/21 10:01:53 - 0:04:27 - Info test message. ForkPoolWorker-40
INFO - 02/26/21 10:01:54 - 0:04:27 - outer
INFO - 02/26/21 10:01:54 - 0:04:27 - Info test message. ForkPoolWorker-42
INFO - 02/26/21 10:01:54 - 0:04:27 - middle


innert:  13%|█▎        | 13/100 [01:07<01:21,  1.07it/s]







middle:  23%|██▎       | 113/500 [01:44<06:02,  1.07it/s]








middle:  23%|██▎       | 113/500 [01:44<05:55,  1.09it/s]









innert:  12%|█▏        | 12/100 [01:06<01:20,  1.10it/s]










innert:  13%|█▎        | 13/100 [01:07<01:21,  1.06it/s]
























                    


                                                
                                                      

                                                   



                                             




                                          





                                       






                                   







                                 








                              









                          










                       












  0%|          | 0/500 [00:26<?, ?it/s]











innert:  13%|█▎        | 13/100 [01:06<01:19,  1.09it/s]


outer:  11%|█▏      

INFO - 02/26/21 10:01:54 - 0:04:27 - outer
INFO - 02/26/21 10:01:54 - 0:04:27 - outer
INFO - 02/26/21 10:01:54 - 0:04:27 - middle
INFO - 02/26/21 10:01:54 - 0:04:27 - middle






middle:  23%|██▎       | 113/500 [01:44<06:08,  1.05it/s]




outer:  11%|█▏        | 113/1000 [01:44<13:34,  1.09it/s]





middle:  22%|██▏       | 112/500 [01:44<05:54,  1.10it/s]






innert:  13%|█▎        | 13/100 [01:07<01:21,  1.07it/s]







middle:  23%|██▎       | 113/500 [01:44<06:02,  1.07it/s]








middle:  23%|██▎       | 113/500 [01:44<05:55,  1.09it/s]









innert:  12%|█▏        | 12/100 [01:06<01:20,  1.10it/s]










innert:  13%|█▎        | 13/100 [01:07<01:21,  1.06it/s]













  0%|          | 0/500 [00:00<?, ?it/s]

KeyError: 9

In [18]:
help(TqdmMultiProcessPool(1).map)

Help on method map in module tqdm_multiprocess.std:

map(global_tqdm, tasks, on_error, on_done) method of tqdm_multiprocess.std.TqdmMultiProcessPool instance



In [50]:
from time import sleep
#from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, freeze_support

def progresser(n):
    # This line is the strange hack
    print(' ', end='', flush=True)

    text = "progresser #{}".format(n)
    for i in notebook.tqdm(range(5000), desc=text, position=n):
         sleep(0.001)
        
freeze_support()
L = list(range(10))
print()
Pool(2).map(progresser, L)

progresser #0:   0%|          | 0/5000 [00:00<?, ?it/s]

progresser #2:   0%|          | 0/5000 [00:00<?, ?it/s]

progresser #1:   0%|          | 0/5000 [00:00<?, ?it/s]

progresser #3:   0%|          | 0/5000 [00:00<?, ?it/s]

progresser #4:   0%|          | 0/5000 [00:00<?, ?it/s]

...

progresser #5:   0%|          | 0/5000 [00:00<?, ?it/s]

progresser #7:   0%|          | 0/5000 [00:00<?, ?it/s]

progresser #8:   0%|          | 0/5000 [00:00<?, ?it/s]

progresser #9:   0%|          | 0/5000 [00:00<?, ?it/s]

[None, None, None, None, None, None, None, None, None, None]

In [49]:
root = os.getcwd()
for f in os.listdir(root):
    path = os.path.join(root, f)
    print(os.path.isdir(path), path)

False /Users/taylora/Documents/Personal/RL/euchre/make_dataset.ipynb
False /Users/taylora/Documents/Personal/RL/euchre/board.py
False /Users/taylora/Documents/Personal/RL/euchre/.DS_Store
True /Users/taylora/Documents/Personal/RL/euchre/thresholds
True /Users/taylora/Documents/Personal/RL/euchre/figs
False /Users/taylora/Documents/Personal/RL/euchre/optimal_strategy.py
True /Users/taylora/Documents/Personal/RL/euchre/__pycache__
False /Users/taylora/Documents/Personal/RL/euchre/test.py
True /Users/taylora/Documents/Personal/RL/euchre/stored_runs
False /Users/taylora/Documents/Personal/RL/euchre/card.py
False /Users/taylora/Documents/Personal/RL/euchre/multiprocessing.ipynb
False /Users/taylora/Documents/Personal/RL/euchre/rule.py
False /Users/taylora/Documents/Personal/RL/euchre/player.py
True /Users/taylora/Documents/Personal/RL/euchre/aggressive
True /Users/taylora/Documents/Personal/RL/euchre/.ipynb_checkpoints
False /Users/taylora/Documents/Personal/RL/euchre/analysis.ipynb


In [1]:
import os
os.listdir()

['make_dataset.ipynb',
 'board.py',
 '.DS_Store',
 'thresholds',
 'figs',
 'optimal_strategy.py',
 'search_lib.py',
 '__pycache__',
 'test.py',
 'stored_runs',
 'card.py',
 'euchre_lib.py',
 'multiprocessing.ipynb',
 'rule.py',
 'player.py',
 'aggressive',
 '.ipynb_checkpoints',
 'analysis.ipynb']

In [23]:
os.path.join(os.getcwd(), os.getcwd()+'/thresholds')

'/Users/taylora/Documents/Personal/RL/euchre/thresholds'

# This works with multiple things writing to the same file at the same time

In [10]:
def testwrite(x):
    print(' ', end='', flush=True)
    for i in notebook.tqdm(range(10)):
        with open('test.txt', 'a') as f:
            f.write('siodf\n')
        time.sleep(0.1)

In [11]:
pool = multiprocessing.Pool(10)
pool.map(testwrite, range(10))
pool.close()
pool.join()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
next((i for i in range(10) if i > 5), None)

6

In [24]:
'abcdef'[-1:]

'f'